### Import necessary Files

In [3]:
from vlac import VLAC
import gensim.models.keyedvectors as word2vec

### Prepare data
In order to prepare the data one must load the corresponding word embeddings either through Gensim or as a simple dictionary (word: vector). It is advised to load only the word embeddings that can be found within the collection of documents as clustering a large number of concepts might be computationally difficult. 

The collection of documents should be represented as a list of strings. 

In [4]:
model_w2v = word2vec.KeyedVectors.load_word2vec_format('Data/web_w2v')
with open('Data/web_docs.txt', "r") as f:
    docs = f.readlines()

### Train model and transform features
The model can be trained using the .fit_transform() procedure. It should be noted that if you use a FastText model, it is advised to set oov to True as it will create word embeddings for out-of-vocabulary words when applying the VLAC procedure (thus, after clustering word embeddings). 

Note: It is advised to save the resulting kmeans model as that will allow you to quickly transform new documents. 

In [ ]:
vlac_model = VLAC(documents=docs, model=model_w2v, oov=False)
vlac_features, kmeans = vlac_model.fit_transform(num_concepts=30)

### Transform features
After having trained the kmeans model, you can use that to quickly transform the features without needing to cluster the embeddings again.  

In [ ]:
vlac_model = VLAC(documents=train_docs, model=model_w2v, oov=False)
test_features = vlac_model.transform(kmeans=kmeans)